In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from scipy import io
import scipy.stats as ss
import numpy as np
import random
from collections import Counter
import csv

In [3]:
def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

def dyad_permutation(dyad_df,n_fold):
    dyad_id = list(dyad_df.Dyad.unique())
    dyad_dict = {}
    # map the dyad_id to subject id
    for d in dyad_id:
        dyad_pair = dyad_df.loc[dyad_df.Dyad==d]
        dyad_dict[d] = list(dyad_pair.index)
    # partition size according to the folds
    parts = int(len(dyad_id)/n_fold)
    dyad_permute = shuffle(dyad_id)
    # get the dyad id in each fold
    
    permute_partitions = list(divide_chunks(dyad_permute,parts))
    trainInd = []
    testInd = []
    # convert dyad it into subject index
    for p in permute_partitions:
        test_ind = []
        for d in p:
            test_ind += dyad_dict[d]
        # train index
        train_ind = list(dyad_df.loc[~dyad_df.index.isin(test_ind)].index)
        testInd.append(test_ind)
        trainInd.append(train_ind)
    return trainInd,testInd

def dyad_permutation_stratified(dyad_df,n_fold):
    dyad_simp = dyad_df.drop_duplicates(subset=['Dyad'])
    dyad_simp = dyad_simp.sort_values(by='Dyad',ascending=True)
    dyad_id = list(dyad_simp.Dyad)
    dyad_label = list(dyad_simp.Label)
    dyad_dict = {}
    # map the dyad_id to subject id
    for d in dyad_id:
        dyad_pair = dyad_df.loc[dyad_df.Dyad==d]
        dyad_dict[d] = list(dyad_pair.index)
    # define the splitter
    kfold = StratifiedKFold(n_splits=n_fold,shuffle=True)
    # get the dyad id in each fold
    trainInd = []
    testInd = []
    trainLabel = []
    testLabel = []
    for train, test in kfold.split(dyad_id,dyad_label):
        train_ind = []
        test_ind = []
        for tr in train:
            train_ind += dyad_dict[dyad_id[tr]]
        for ts in test:
            test_ind += dyad_dict[dyad_id[ts]]
        train_labels = list(dyad_df.loc[dyad_df.index.isin(train_ind)].Label)    
        test_labels = list(dyad_df.loc[dyad_df.index.isin(test_ind)].Label)
        one_count = test_labels.count(1)
        if one_count == len(test_labels):
            print (test_labels)
        trainInd.append(train_ind)
        testInd.append(test_ind)
        trainLabel.append(train_labels)
        testLabel.append(test_labels)
    return trainInd,testInd,trainLabel,testLabel

def permutation_stratified(df,n_fold):
     # define the splitter
    kfold = StratifiedKFold(n_splits=n_fold,shuffle=True)
    ids = list(df.index)
    labels = list(df.Label)
    trainInd = []
    testInd = []
    trainLabel = []
    testLabel = []
    for train, test in kfold.split(ids,labels):
        train_ind = []
        test_ind = []
        for tr in train:
            train_ind.append(ids[tr])
        for ts in test:
            test_ind.append(ids[ts])
        train_labels = list(df.loc[df.index.isin(train_ind)].Label)    
        test_labels = list(df.loc[df.index.isin(test_ind)].Label)
        trainInd.append(train_ind)
        testInd.append(test_ind)
        trainLabel.append(train_labels)
        testLabel.append(test_labels)
    return trainInd,testInd,trainLabel,testLabel

### Load data

In [4]:
folder = '/Users/senadong/Documents/LMU_works/Jana/cv_customisation_paper2/stratified_cv_structure/'

**Label encoding**<br>
1: ASD, 2: CC

In [4]:
dyad_file = folder + 'idlabel_personbased.xlsx'
dyad_df = pd.read_excel(dyad_file,engine='openpyxl')
# set index to start with 1 to be compatible with matlab
dyad_df.index += 1 
# make numerical label
dyad_df['Label'] = 1
dyad_df.loc[dyad_df['Diagnosis']=='CC','Label'] = 2
display(dyad_df.head())
display(dyad_df.shape)

,Id,Diagnosis,Dyad,Label
1,100_diagnostician,ASD,100,1
2,100_patient,ASD,100,1
3,101_diagnostician,ASD,101,1
4,101_patient,ASD,101,1
5,103_diagnostician,ASD,103,1


(188, 4)

### Create Dyad-based CV structure

In [5]:
cv2 = 10
p2 = 10
cv1 = 10
p1 = 1

In [16]:
cv2_train = []
cv2_test = []
cv2_train_label = []
cv2_test_label = []
cv1_all = []
cv1_labels = []

for p in range(p2):
    train2,test2,train_label2,test_label2 = dyad_permutation_stratified(dyad_df,cv2)
    cv2_train.append(train2)
    cv2_test.append(test2)
    cv2_train_label.append(train_label2)
    cv2_test_label.append(test_label2)
    
    cv1_col = []
    for t in train2:
        df_temp = dyad_df.loc[dyad_df.index.isin(t)]
        df_temp = df_temp.reset_index()
        df_temp.index += 1 
        #display(df_temp.head())
        cv1_dict = {}
        cv1_tr = []
        cv1_ts = []
        cv1_tr_label = []
        cv1_ts_label = []
        
        for p_in in range(p1):
            train1,test1,train_label1,test_label1 = dyad_permutation_stratified(df_temp,cv1)
            cv1_tr.append(train1)
            cv1_ts.append(test1)
            cv1_tr_label.append(train_label1)
            cv1_ts_label.append(test_label1)
        cv1_dict['TrainInd'] = cv1_tr
        cv1_dict['TestInd'] = cv1_ts
        cv1_dict['TrainLabel'] = cv1_tr_label
        cv1_dict['TestLabel'] = cv1_ts_label
        cv1_col.append(cv1_dict)
        
    cv1_all.append(cv1_col)

cv1_all = np.array(cv1_all, dtype=object)
cv2_train = np.array(cv2_train, dtype=object)
cv2_test = np.array(cv2_test, dtype=object)

print ('save cv2')
io.savemat(folder+'TrainInd.mat', {'Train_cv2_raw': cv2_train})
io.savemat(folder+'TestInd.mat', {'Test_cv2_raw': cv2_test})
io.savemat(folder+'TrainLabel.mat', {'Train_cv2_label': cv2_train_label})
io.savemat(folder+'TestLabel.mat', {'Test_cv2_label': cv2_test_label})
print (cv2_train.shape)
print (cv2_test.shape)

print ('save cv1')
print (cv1_all.shape)
io.savemat(folder+'cvin.mat', {'cv1_all': cv1_all})

save cv2
(10, 10)
(10, 10)
save cv1
(10, 10)


/Users/senadong/miniconda3/envs/rosetta_base/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


### Create CV structure

**Label encoding**<br>
1: ASD, 2: CC

In [17]:
participants_file = folder + 'idlabel_dyadbased.xlsx'
participants_df = pd.read_excel(participants_file,engine='openpyxl')
# set index to start with 1 to be compatible with matlab
participants_df.index += 1 
# make numerical label
participants_df['Label'] = 1
participants_df.loc[participants_df['Diagnosis']=='CC','Label'] = 2
participants_df.head()

,Id,Diagnosis,Label
1,SNU100,ASD,1
2,SNU101,ASD,1
3,SNU103,ASD,1
4,SNU105,ASD,1
5,SNU106,CC,2


In [18]:
cv2 = 10
p2 = 10
cv1 = 10
p1 = 1

In [30]:
cv2_train = []
cv2_test = []
cv2_train_label = []
cv2_test_label = []
cv1_all = []
cv1_labels = []

for p in range(p2):
    train2,test2,train_label2,test_label2 = permutation_stratified(participants_df,cv2)
    cv2_train.append(train2)
    cv2_test.append(test2)
    cv2_train_label.append(train_label2)
    cv2_test_label.append(test_label2)
    
    cv1_col = []
    for t in train2:
        df_temp = participants_df.loc[participants_df.index.isin(t)]
        df_temp = df_temp.reset_index()
        df_temp.index += 1 
        #display(df_temp.head())
        cv1_dict = {}
        cv1_tr = []
        cv1_ts = []
        cv1_tr_label = []
        cv1_ts_label = []
        
        for p_in in range(p1):
            train1,test1,train_label1,test_label1 = permutation_stratified(df_temp,cv1)
            cv1_tr.append(train1)
            cv1_ts.append(test1)
            cv1_tr_label.append(train_label1)
            cv1_ts_label.append(test_label1)
        cv1_dict['TrainInd'] = cv1_tr
        cv1_dict['TestInd'] = cv1_ts
        cv1_dict['TrainLabel'] = cv1_tr_label
        cv1_dict['TestLabel'] = cv1_ts_label
        cv1_col.append(cv1_dict)
        
    cv1_all.append(cv1_col)

cv1_all = np.array(cv1_all, dtype=object)
cv2_train = np.array(cv2_train, dtype=object)
cv2_test = np.array(cv2_test, dtype=object)

print ('save cv2')
io.savemat(folder+'TrainInd.mat', {'Train_cv2_raw': cv2_train})
io.savemat(folder+'TestInd.mat', {'Test_cv2_raw': cv2_test})
io.savemat(folder+'TrainLabel.mat', {'Train_cv2_label': cv2_train_label})
io.savemat(folder+'TestLabel.mat', {'Test_cv2_label': cv2_test_label})
print (cv2_train.shape)
print (cv2_test.shape)

print ('save cv1')
print (cv1_all.shape)
io.savemat(folder+'cvin.mat', {'cv1_all': cv1_all})

save cv2
(10, 10)
(10, 10)
save cv1
(10, 10)


/Users/senadong/miniconda3/envs/rosetta_base/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


### NDD CV structure
1: NDD 2: CC

In [5]:
ndd_file = folder + 'NDD_idlabel.xlsx'
ndd_df = pd.read_excel(ndd_file,engine='openpyxl')
# set index to start with 1 to be compatible with matlab
ndd_df.index += 1 
# make numerical label
ndd_df['Label'] = 1
ndd_df.loc[ndd_df['Diagnosis']=='CC','Label'] = 2
ndd_df.head()

,Id,Diagnosis,Label
1,SNU100,NDD,1
2,SNU101,NDD,1
3,SNU103,NDD,1
4,SNU105,NDD,1
5,SNU106,NDD,1


In [7]:
cv2 = 10
p2 = 10
cv1 = 10
p1 = 1

In [8]:
cv2_train = []
cv2_test = []
cv2_train_label = []
cv2_test_label = []
cv1_all = []
cv1_labels = []

for p in range(p2):
    train2,test2,train_label2,test_label2 = permutation_stratified(ndd_df,cv2)
    cv2_train.append(train2)
    cv2_test.append(test2)
    cv2_train_label.append(train_label2)
    cv2_test_label.append(test_label2)
    
    cv1_col = []
    for t in train2:
        df_temp = ndd_df.loc[ndd_df.index.isin(t)]
        df_temp = df_temp.reset_index()
        df_temp.index += 1 
        #display(df_temp.head())
        cv1_dict = {}
        cv1_tr = []
        cv1_ts = []
        cv1_tr_label = []
        cv1_ts_label = []
        
        for p_in in range(p1):
            train1,test1,train_label1,test_label1 = permutation_stratified(df_temp,cv1)
            cv1_tr.append(train1)
            cv1_ts.append(test1)
            cv1_tr_label.append(train_label1)
            cv1_ts_label.append(test_label1)
        cv1_dict['TrainInd'] = cv1_tr
        cv1_dict['TestInd'] = cv1_ts
        cv1_dict['TrainLabel'] = cv1_tr_label
        cv1_dict['TestLabel'] = cv1_ts_label
        cv1_col.append(cv1_dict)
        
    cv1_all.append(cv1_col)

cv1_all = np.array(cv1_all, dtype=object)
cv2_train = np.array(cv2_train, dtype=object)
cv2_test = np.array(cv2_test, dtype=object)

print ('save cv2')
io.savemat(folder+'TrainInd.mat', {'Train_cv2_raw': cv2_train})
io.savemat(folder+'TestInd.mat', {'Test_cv2_raw': cv2_test})
io.savemat(folder+'TrainLabel.mat', {'Train_cv2_label': cv2_train_label})
io.savemat(folder+'TestLabel.mat', {'Test_cv2_label': cv2_test_label})
print (cv2_train.shape)
print (cv2_test.shape)

print ('save cv1')
print (cv1_all.shape)
io.savemat(folder+'cvin.mat', {'cv1_all': cv1_all})

save cv2
(10, 10)
(10, 10)
save cv1
(10, 10)


/Users/senadong/miniconda3/envs/rosetta_base/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
